In [1]:
import functools
import operator

import z3

In [2]:
ROW_PRODS = [294, 216, 135, 98, 112, 84, 245, 40]
COL_PRODS = [8_890_560, 156_800, 55_566]

In [3]:
solver = z3.Solver()

In [4]:
matrix = []
for i in range(len(ROW_PRODS)):
    for j in range(len(COL_PRODS)):
        cell = z3.Int(f"{i}_{j}")
        solver.add(cell >= 0)
        solver.add(cell <= 9)
        if j == 0:
            matrix.append([])
        matrix[i].append(cell)

In [5]:
# no numpy slicing :(

# set the row constraints to equal the horizontal product
for row_i, row_prod in enumerate(ROW_PRODS):
    cur_row = []
    for col_j, col_prod in enumerate(COL_PRODS):
        cur_row.append(matrix[row_i][col_j])
    solver.add(z3.Product(cur_row) == row_prod)

In [6]:
# # set the column constraints to equal vertical product
for col_j, col_prod in enumerate(COL_PRODS):
    cur_col = []
    for row_i, row_prod in enumerate(ROW_PRODS):
        cur_col.append(matrix[row_i][col_j])
    solver.add(z3.Product(cur_col) == col_prod)

In [7]:
%%time
solver.check()

CPU times: user 5.07 s, sys: 32.2 ms, total: 5.1 s
Wall time: 5.1 s


sat

In [8]:
model = solver.model()

In [9]:
for i in range(len(ROW_PRODS)):
    rows = []
    for j in range(len(COL_PRODS)):
        # extracting values from model as python objects
        result = str(model[matrix[i][j]])
        rows.append(result)
    # too elaborate printing and manual validation
    product = functools.reduce(operator.mul, map(int, rows))
    print(f"{'-'.join(rows)}")
    print(f"\t{ ROW_PRODS[i] } { ROW_PRODS[i] == product } { product }")

7-7-6
	294 True 294
9-8-3
	216 True 216
9-5-3
	135 True 135
7-2-7
	98 True 98
8-2-7
	112 True 112
7-4-3
	84 True 84
5-7-7
	245 True 245
8-5-1
	40 True 40
